# **Data scraping of spotify artists data**

### **Lyrics extractor**

This is a function built to extract lyrics from the "Genius" website. Inputs are the song title and artist names both as string, and the output is the lyrics as a string.

REQUIREMENTS:

You must go to the [Genius API website](https://genius.com/api-clients) and create a "New API Client". Once you have that, you just generate an acess token to paste into the variable in the code cell below.

In [27]:
import requests
from bs4 import BeautifulSoup
import re

GENIUS_TOKEN = "Lb6LyUbQMTyW3iPT5BkepQzUvTG5iUkY0gWzdeFMYiSq8lbREvcyi2klHVPiZiSr"

In [41]:
def get_lyrics(song_title, artist_name):
    base_url = "https://api.genius.com/search"
    headers = {"Authorization": f"Bearer {GENIUS_TOKEN}"}
    params = {"q": f"{song_title} {artist_name}"}
    response = requests.get(base_url, headers=headers, params=params).json()

    hits = response["response"]["hits"]
    if not hits:
        print("No results found.")
        return None

    # Pick first correct artist result
    for hit in hits:
        result = hit["result"]
        if result["primary_artist"]["name"].lower() == artist_name.lower():
            song_url = result["url"]
            break
    else:
        song_url = hits[0]["result"]["url"]

    # Scrape the lyrics page
    page = requests.get(song_url)
    soup = BeautifulSoup(page.text, "html.parser")

    # Extract lyrics
    lyrics_divs = soup.select("div[data-lyrics-container='true']")
    if lyrics_divs:
        lyrics = "\n".join(div.get_text(separator="\n") for div in lyrics_divs)
    else:
        candidates = soup.find_all("div", class_=re.compile("(Lyrics__Root|Lyrics-sc-|SongPage__Section)"))
        lyrics = "\n".join(c.get_text(separator="\n") for c in candidates) if candidates else ""

    # Trim off everything above the first [Verse 1]
    match = re.search(r"\[verse\s*1\]", lyrics, flags=re.IGNORECASE)
    if match:
        lyrics = lyrics[match.start():]   # keep from [Verse 1] onward
    
    # Remove everything in brackets (like [Chorus], [Bridge] and such)
    lyrics = re.sub(r"\[.*?\]", "", lyrics)

    # Optional: stop at footer sections
    lyrics = re.split(r"You might also like|Embed|See.*Lyrics$", lyrics)[0].strip()

    return lyrics if lyrics else None

Testing below.

In [ ]:
print(get_lyrics("", "Chappell Roan"))

It's fine, it's cool
You can say that we are nothing, but you know the truth
And guess I'm the fool
With her arms out like an angel through the car sunroof

I don't wanna call it off
But you don't wanna call it love
You only wanna be the one that I call "baby"

You can kiss a hundred boys in bars
Shoot another shot, try to stop the feeling
You can say it's just the way you are
Make a new excuse, 'nother stupid reason
Good luck, babe (Well, good luck)
Well, good luck, babe (Well, good luck)
You'd have to stop the world just to stop the feeling
Good luck, babe (Well, good luck)
Well, good luck, babe (Well, good luck)
You'd have to stop the world just to stop the feeling

I'm cliché, who cares?
It's a sexually explicit kind of love affair
And I cry, it's not fair
I just need a little lovin', I just need a little air


Think I'm gonna call it off
Even if you call it love
I just wanna love someone who calls me "baby"

You can kiss a hundred boys in bars
Shoot another shot, try to stop the f